In [1]:
import pandas as pd
import numpy as np
import os

from collections import defaultdict

In [2]:
dir_data = "./data"
data_file = "raspberry_praline_composition.csv"

recipe_raw = pd.read_csv(os.path.join(dir_data, data_file))
display(recipe_raw)

,Component,Qty,Level
0,Chocolate Praline,100.000000,1
1,Dark Chocolate 55% Cocoa,55.000000,2
2,Cocoa Liquor,22.000000,3
3,Cocoa Butter,5.500000,3
4,Sugar,24.200000,3
5,Refined Cane Sugar,24.197250,4
6,Anti-Caking Agent,0.002750,4
7,Alkalised Cocoa Powder,2.750000,3
8,Cocoa Powder,2.736250,4
9,Potassium Carbonate,0.013750,4


In [3]:
from collections import defaultdict
import numpy as np
import pandas as pd

class Recipe:

    def __init__(self, df: pd.DataFrame, params: dict = None, exclude_highest_level = "auto", name: str = None):
        
        self.df_ = df
        self.df_.columns = [x.strip() for x in self.df_.columns]
        
        if params is None:
            self.params = {"ing": "Component", "qty": "Qty", "lvl": "Level"}
        else:
            self.params = params

        self.ing_key, self.qty_key, self.lvl_key = list(self.params.keys())[:3]
        self.ing_col, self.qty_col, self.lvl_col = list(self.params.values())[:3]
        
        if exclude_highest_level == "auto":
            pass
            
        self.highest_level = self.df_[self.lvl_col].min()
        self.recipe_ = self.df_[self.df_[self.lvl_col] > 1].reset_index(drop=True)
       
        if name is None:
            self.name = df.loc[df[self.lvl_col] == self.highest_level, self.ing_col][0]
        else:
            self.name = name
        
        self.rec_dict = defaultdict(self.dict_template)
   
    def dict_template(self):
       return {key: [] for key in self.params.keys()}
   
    def copy_to_dict(self, idx, level):
        for key, colname in self.params.items():
            self.rec_dict[level][key].append(self.recipe_.loc[idx][colname])
    
    def __repr__(self):
        return f"Recipe(df)"
    
    def __str__(self):
        n_components = self.recipe_.shape[0]
        return "{} recipe with {} components".format(self.name, n_components)
    
    def traverse_recipe(self, idx = 0, start_level = 1, max_level = np.inf):

        current_level = self.recipe_.loc[idx, self.lvl_col]
            
        if (idx == self.recipe_.index.max()):
            previous_level = self.recipe_.loc[idx-1, self.lvl_col]
            
            if (current_level <= max_level) and (previous_level >= current_level):
                self.copy_to_dict(idx, level = max_level)
                return None # default_dict
            
            elif (idx == self.recipe_.index.max()) and (current_level > max_level):
                return None # default_dict
            
        else:
            next_level = self.recipe_.loc[(idx+1), self.lvl_col]
            
            if ~(next_level <= current_level) and ~(current_level > max_level) and (next_level > max_level):
                self.copy_to_dict(idx, level = max_level)
                self.traverse_recipe(idx + 1, start_level, max_level)
                
            elif (next_level <= current_level) and ~(current_level > max_level) and ~(next_level > max_level):
                self.copy_to_dict(idx, level = max_level)
                self.traverse_recipe(idx + 1, start_level, max_level)
                
            else:
                self.traverse_recipe(idx + 1, start_level, max_level)

    def extract_recipe(self, max_level = np.inf, aggregate = False, aggregate_by = None, sort = True, sort_by = None, ascending = False):

        if max_level in self.rec_dict.keys():
            res = pd.DataFrame(self.rec_dict[max_level])
        else:
            self.traverse_recipe(idx = 0, max_level = max_level)
            res = pd.DataFrame(self.rec_dict[max_level])
            
        if aggregate:
            if aggregate_by is None:
                aggregate_by = self.ing_key

            res = res.groupby(aggregate_by).sum().reset_index()[[self.ing_key, self.qty_key]]
            if sort:
                if sort_by is None:
                    sort_by = self.qty_key
        
                res = res.sort_values(sort_by, ascending = ascending).reset_index(drop=True)

        return res
    
    @property
    def df(self):
        return self.df_
    
    @property
    def recipe(self):
        return self.recipe_
    
    @classmethod
    def from_csv(cls, filepath: str, *args, **kwargs):
        df = pd.read_csv(filepath, *args, **kwargs)
        return cls(df)

    @classmethod
    def from_excel(cls, filepath: str, *args, **kwargs):
        df = pd.read_excel(filepath, *args, **kwargs)
        return cls(df)

In [4]:
rec = Recipe(recipe_raw)
print(rec)
print(repr(rec))

print("Highest Level:", rec.highest_level)
level_count = rec.df.groupby("Level").count()["Component"]
print(level_count)

Chocolate Praline recipe with 28 components
Recipe(df)
Highest Level: 1
Level
1     1
2     3
3    10
4    13
5     2
Name: Component, dtype: int64


In [5]:
recipe_flat = rec.extract_recipe(4, aggregate=True, sort=True)
display(recipe_flat)

,ing,qty
0,Refined Cane Sugar,24.197250
1,Cocoa Liquor,22.000000
2,Sugar,19.333333
3,Hazelnut Paste,12.566667
4,Palm Oil,8.699782
5,Cocoa Butter,5.500000
6,Roasted Diced Hazelnut Pieces,2.900000
7,Cocoa Powder,2.736250
8,Freeze-dried Raspberry Pieces,1.500000
9,Carrier,0.269500
